In [ ]:
from PIL import Image 
import os
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
import json
import cv2
import random
import glob
import math

In [ ]:
def mask_to_contour(mask):
    contours, _ = cv2.findContours(
        mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
    )
    return contours

In [ ]:
size = 512 #choose size of tile

im_path = r"" #path to original images
mask_path = r"" #path to original masks
tiled_im_path = r"" #path to tiled images
tiled_mask_path = r"" #path to tiled masks
tiled_ann_path =  r"" #path to tiled/rotated json

# Randomly tile 1 image

In [ ]:
file_name = "" #choose image
imPath = os.path.join(im_path,file_name)
image = cv2.imread(imPath)
plt.imshow(image)

In [ ]:
shape = image.shape
height = shape[0]
width = shape[1]

height_new = math.ceil(height/size) * size
height_scale = height_new/height 
delta_y= height - height_new
    
width_new = math.ceil(width/size) * size
width_scale = width_new/ width
delta_x = width - width_new

resized_im = cv2.resize(image, (width_new, height_new), cv2.INTER_LINEAR)

plt.imshow(resized_im)

In [ ]:
rotations = [0,15,30,45,60,90,180,270]

x_start = random.randint(0, resized_im.shape[1] - size)
y_start = random.randint(0, resized_im.shape[0] - size)
            
cropped_im = resized_im[y_start:y_start+size, x_start:x_start+size]
    
rot = rotations[random.randrange(0,len(rotations))]
            
w = cropped_im.shape[0]
h = cropped_im.shape[1]
x = w//2
y = h//2

M = cv2.getRotationMatrix2D((x, y), rot,1.0)
cos = np.abs(M[0, 0])
sin = np.abs(M[0, 1])
nW = int((h * sin) + (w * cos))
nH = int((h * cos) + (w * sin))
M[0, 2] += (nW / 2) - x
M[1, 2] += (nH / 2) - y
        
rotated_im = cv2.warpAffine(cropped_im, M, (nW, nH))
            
if (rotated_im.shape[0] != 512):
        mid_x, mid_y = int(rotated_im.shape[0]/2), int(rotated_im.shape[1]/2)
        cw2, ch2 = int(512/2), int(512/2) 
        rotated_im = rotated_im[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
      


In [ ]:
plt.imshow(cropped_im)

In [ ]:
plt.imshow(rotated_im)
print(rot)

# Tile all images

In [ ]:
for ii in os.listdir(mask_path):
    
    file_name = os.path.basename(ii)
    imPath= os.path.join(im_path,file_name)
    maskPath = os.path.join(mask_path,file_name)
    image = cv2.imread(imPath, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(maskPath, cv2.IMREAD_GRAYSCALE)
    
        
    height, width= mask.shape
    
    height_new = math.ceil(height/size) * size
    height_scale = height_new/height 
    delta_y= height - height_new
    
    width_new = math.ceil(width/size) * size
    width_scale = width_new/ width
    delta_x = width - width_new
    
    resized_im = cv2.resize(image, (width_new, height_new), cv2.INTER_LINEAR)
    resized_mask = cv2.resize(mask,(width_new, height_new), cv2.INTER_LINEAR)
    
    tiled_ims_list = []
    
    for i in range(height_new//size):
        for j in range(width_new//size):
            
            tile = resized_im[i*size : (i+1)*size, j*size : (j+1)*size]
            
            tile_mask = resized_mask[i*size : (i+1)*size, j*size : (j+1)*size]
            
            mask = tile_mask.astype(np.uint8)*255
    
            if np.any(mask):
                    
                tile_name = file_name.split(".")[0] + "_" + str(i) + "_" + str(j) + ".png" 
                tiled_ims_list.append(tile_name)
                
                new_path_im = os.path.join(tiled_im_path, tile_name)
                cv2.imwrite(new_path_im, tile)
           
                new_path_mask = os.path.join(tiled_mask_path, tile_name)
                cv2.imwrite(new_path_mask, tile_mask)
    
   

# Randomly tile and rotate all images

In [ ]:
rotations = [0,15,30,45,60,90,180,270]

for ii in os.listdir(mask_path):
    
    file_name = os.path.basename(ii)
    imPath= os.path.join(im_path,file_name)
    maskPath = os.path.join(mask_path,file_name)
    image = cv2.imread(imPath, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(maskPath, cv2.IMREAD_GRAYSCALE)
    
    height, width= mask.shape
    
    height_new = math.ceil(height/size) * size
    height_scale = height_new/height 
    delta_y= height - height_new
    
    width_new = math.ceil(width/size) * size
    width_scale = width_new/ width
    delta_x = width - width_new

    resized_im = cv2.resize(image, (width_new, height_new), cv2.INTER_LINEAR)
    resized_mask = cv2.resize(mask,(width_new, height_new), cv2.INTER_LINEAR)
    
    for i in range(0,4): #select 4 random tiles from the image
            
        x_start = random.randint(0, resized_mask.shape[1] - size)
        y_start = random.randint(0, resized_mask.shape[0] - size)
            
        cropped_im = resized_im[y_start:y_start+size, x_start:x_start+size]
        cropped_mask = resized_mask[y_start:y_start+size, x_start:x_start+size]
            
        rot = rotations[random.randrange(0,len(rotations))]
            
        w = cropped_im.shape[0]
        h = cropped_im.shape[1]
        x = w//2
        y = h//2

        M = cv2.getRotationMatrix2D((x, y), rot,1.0)
        cos = np.abs(M[0, 0])
        sin = np.abs(M[0, 1])
        nW = int((h * sin) + (w * cos))
        nH = int((h * cos) + (w * sin))
        M[0, 2] += (nW / 2) - x
        M[1, 2] += (nH / 2) - y
        
        rotated_im = cv2.warpAffine(cropped_im, M, (nW, nH))
        rotated_mask = cv2.warpAffine(cropped_mask, M, (nW, nH))
            
        if (rotated_im.shape[0] != 512):
            mid_x, mid_y = int(rotated_im.shape[0]/2), int(rotated_im.shape[1]/2)
            cw2, ch2 = int(512/2), int(512/2) 
            rotated_im = rotated_im[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
            rotated_mask = rotated_mask[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
    
        if np.any(rotated_mask):
                    
            tile_name = file_name.split(".")[0] + "_" + str(y_start) + "_" + str(x_start) + "_" + str(rot) + ".png" 
                
            new_path_im = os.path.join(tiled_im_path, tile_name)
            cv2.imwrite(new_path_im, rotated_im)
           
            new_path_mask = os.path.join(tiled_mask_path, tile_name)
            cv2.imwrite(new_path_mask, rotated_mask)
    
   

# Save to json

In [ ]:
CATEGORY_TEMPLATE = {
    "id": 1,
    "name": "person",
    "supercategory": None,
    "metadata": {},
    "color": "#69db90",
}

IMAGES_TEMPLATE = {
    "id": 0,
    "width": 530,
    "height": 301,
    "file_name": "",
    "path": "",
    "license": None,
    "fickr_url": None,
    "coco_url": None,
    "date_captured": None,
    "metadata": {},
}

ANNOTATIONS_TEMPLATE = {
    "id": 1,
    "image_id": 0,
    "category_id": 1,
    "width": 530,
    "height": 301,
    "area": 59767,
    "segmentation": [[221, 7, 220, 8, 215, 8]],
    "bbox": [150, 7, 253, 286],
    "metadata": {},
    "color": "#bc07ae",
    "iscrowd": 0,
}

In [ ]:
def mask_to_contour(mask):
    contours, _ = cv2.findContours(
        mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
    )
    return contours

def get_random_color():
    return ["#" + "".join([random.choice("ABCDEF0123456789") for _ in range(6)])][
        0
    ].lower()

class dotdict(dict):
    """dot.notation access to dictionary attributes"""

    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    

In [ ]:
categories = []

category = dotdict(CATEGORY_TEMPLATE)
category.id = 1
category.name = 'root'
category.color = get_random_color()

categories += [category]

annotations = []
images = []
path = tiled_mask_path + "\*.png"
im_num = 1
ann_num = 1 

for img in glob.glob(path):
    im = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    im = im.astype(np.uint8) * 255
    
    image = dotdict(IMAGES_TEMPLATE)
    image.width = im.shape[1]
    image.height = im.shape[0]
    image.file_name = os.path.basename(img)
    image.id = im_num
    images += [image]
    
    contours = mask_to_contour(im)
    
    for contour in contours:
        
        bbox = cv2.boundingRect(contour)
        segmentation = np.array(contour).flatten().tolist()
        
        if (len(segmentation) == 4):
                segmentation.append(segmentation[-2])
                segmentation.append(segmentation[-2])
                print(segmentation)
                
         
        if (len(segmentation) < 4):
            segmentation.append(segmentation[-2])
            segmentation.append(segmentation[-2])
            segmentation.append(segmentation[-2])
            segmentation.append(segmentation[-2])
            print(segmentation)
        
        
        annotation = dotdict(ANNOTATIONS_TEMPLATE)
        annotation.id = ann_num
        annotation.image_id = im_num
        annotation.category_id = 1
        annotation.width = bbox[2]
        annotation.height = bbox[3]
        annotation.area = cv2.contourArea(contour)
        annotation.segmentation = [segmentation]
        annotation.bbox = [bbox[0],bbox[1],bbox[2],bbox[3]]
        annotation.color = get_random_color()
        
        annotations += [annotation]
        
        ann_num = ann_num + 1
    
    im_num = im_num + 1 
    
coco = {"annotations": annotations, "categories": categories, "images": images}

In [ ]:
with open(tiled_ann_path, "w") as outfile:
    json.dump(coco, outfile)
outfile.close()

In [ ]:
#inspect to make sure everything looks okay :D

coco_path = tiled_ann_path
coco = COCO(annotation_file=coco_path)
cat_ids = coco.getCatIds()

print(f"Number of Unique Categories: {len(cat_ids)}")
print("Category IDs:")
print(cat_ids)
cats = coco.loadCats(cat_ids)
cat_names = [cat["name"] for cat in cats]
print("Categories Names:")
print(cat_names)

with open(coco_path) as jsonFile:
    cocoAnns = json.load(jsonFile)
jsonFile.close()

images = cocoAnns['images']
annotations = cocoAnns['annotations']

print("num annotations:", len(annotations))

In [ ]:
img_ids = coco.getImgIds(catIds=1)
print(f"Number of Images Containing root: {len(img_ids)}")

In [ ]:
img_id = img_ids[2500] #choose image ID
img_info = coco.loadImgs([img_id])[0]
img_file_name = img_info["file_name"]

print(
    f"Image ID: {img_id}, File Name: {img_file_name}"
)

ann_ids = coco.getAnnIds(imgIds= [img_id], catIds = [1], iscrowd=None)
print(ann_ids)
anns = coco.loadAnns(ann_ids)
print(f"Annotations for Image ID {img_id}:")
print(anns)

In [ ]:
im_path = os.path.join(tiled_im_path,img_file_name)
im = Image.open(im_path)
plt.figure(figsize=(20, 20))
plt.imshow(np.asarray(im), cmap='gray')
coco.showAnns(anns)
print(im_path)

In [ ]:
im_path = os.path.join(tiled_im_path,img_file_name)
im = Image.open(im_path)
plt.figure(figsize=(20, 20))
plt.imshow(np.asarray(im), cmap='gray')